### Анна Тимошенко

In [1]:
import pandas as pd
import pandahouse as ph
from scipy.stats import ttest_ind
import seaborn as sns
import numpy as np

In [2]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'simulator_20220820',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-07-13' and '2022-07-19'
    and exp_group in (0,1,2,3)
GROUP BY exp_group, user_id
"""

df = ph.read_clickhouse(q, connection=connection)

Считаем общий CTR в контрольной группе  𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠) 

Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠 

После чего сравним  t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠

### Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [4]:
ctr_control0 = df[df.exp_group == 0].likes.sum() / df[df.exp_group == 0].views.sum()
df.loc[df.exp_group.isin([0,3]), 'linearized_likes'] = df[df.exp_group.isin([0,3])].apply(
    lambda x: x['likes'] - ctr_control0 * x['views'], axis=1)

In [5]:
ttest_ind(df.loc[df.exp_group == 0, 'linearized_likes'],
          df.loc[df.exp_group == 3, 'linearized_likes'],
          equal_var=False)

Ttest_indResult(statistic=-15.214995460903827, pvalue=5.4914249479690016e-52)

In [6]:
ttest_ind(df[df.exp_group == 0].ctr,
          df[df.exp_group == 3].ctr,
          equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

Отличие есть, p-value меньше не стало

### Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [7]:
ctr_control1 = df[df.exp_group == 1].likes.sum() / df[df.exp_group == 1].views.sum()
df.loc[df.exp_group.isin([1,2]), 'linearized_likes'] = df[df.exp_group.isin([1,2])].apply(
    lambda x: x['likes'] - ctr_control1 * x['views'], axis=1)

In [8]:
ttest_ind(df.loc[df.exp_group == 1, 'linearized_likes'],
          df.loc[df.exp_group == 2, 'linearized_likes'],
          equal_var=False)

Ttest_indResult(statistic=6.122579994775974, pvalue=9.439432187037644e-10)

In [9]:
ttest_ind(df[df.exp_group == 1].ctr,
          df[df.exp_group == 2].ctr,
          equal_var=False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

Отличие есть, p-value стало меньше. Это показывает, что t-test на линеаризованных лайках более эффективен.